In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../data/irtm.csv')

In [ ]:
df.head()

In [ ]:
print(df.shape)
df['object_class'].unique()

In [ ]:
df2 = df[(df['object_class']!='thaumiel') & (df['object_class']!='neutralized')]

In [ ]:
df[[len(str(c)) < 100 for c in df['content']]]
df = df[np.array([len(str(c)) >= 100 for c in df['content']]) & np.array([cls in ['euclid', 'keter', 'safe'] for cls in df['object_class']])].reset_index(drop=True)
df.head()

In [ ]:
adjacency_list = [ [] for i in range(df.shape[0])]

for row in range(df.shape[0]):
    
    df['content'][row] = df['content'][row].replace('\n',' ')
    tokens = df['content'][row].split()
    
    for ID in range(df.shape[0]):
        if(df['title'][ID] in tokens and ID != row):
            adjacency_list[row].append(ID)
    
    #print(adjacency_list[row])

In [ ]:
count = 0
for a in adjacency_list:
    if(a != []):
        print(a)
        count += 1
print(count)

In [ ]:
for i in range(len(adjacency_list)):
    adjacency_list[i] = list(adjacency_list[i])

In [ ]:
adjacency_list_correct = []
startPoint = []
count = 0

for i in range(len(adjacency_list)):
    if(adjacency_list[i] != []):
        a = adjacency_list[i].copy()
        for node in a:
            pair = (i,node)
            startPoint.append(i)
            adjacency_list_correct.append(pair)

In [ ]:
adjacency_list_correct

In [ ]:
import networkx as nx
G=nx.Graph()
G.add_nodes_from(startPoint)
G.add_edges_from(adjacency_list_correct)
print("Nodes of graph: ")
print(G.nodes())
print("Edges of graph: ")
print(G.edges())

In [ ]:
nx.draw(G,node_size = 10)
plt.savefig("simple_path.png") # save as png
plt.show() # display

In [ ]:
ccs = list(nx.connected_components(G))
f = open('connected components.txt','w')
for cc in ccs:
    cc = list(cc)
    if len(cc) > 2:
        for c in cc:
            f.write(str(c)+' ')
        f.write('\n')
f.close()

In [ ]:
from pa2 import make_dict
# dic = make_dict(df['content'])

In [ ]:
from pa2 import save_dict
# save_dict(dic, 'dictionary.txt')

### Create vectors

In [ ]:
from pa2 import create_vector
# create_vector(df['content'], list(df.index))

### Get dataset

In [ ]:
from pa3_NB import *

In [ ]:
fact = pd.factorize(df.object_class)

In [ ]:
df.object_class

In [ ]:
plt.hist(df['rating'])

In [ ]:
df.describe()

In [ ]:
df['adj_rating'] = df['rating']**(1/5)

In [ ]:
df['adj_rating'][1]

In [ ]:
rating_class = []
for i in range(len(df['rating'])):
    if df['rating'][i] <= 60 :
        rating_class.append('class_1')
    elif df['rating'][i] <= 112 :
        rating_class.append('class_2')
    elif df['rating'][i] <= 197 :
        rating_class.append('class_3')
    else:
        rating_class.append('class_4')

print(len(rating_class))    

In [ ]:
sum(np.array(rating_class) == 'class_4')

In [ ]:
fact2 = pd.factorize(rating_class)
print(fact2)

In [ ]:
ind = sorted(list(set(np.arange(5739)) - set([36,
96,
787,
1113,
1703,
1753,
2596,
2642,
3090,
4072,
4105,
4238,
5253,
5667])))
dataset = Dataset(np.array(ind), fact2[0][ind]+1)

In [ ]:
len(dataset)

In [ ]:
terms = read_dict(Config.dictionary_file)

In [ ]:
plt.hist([df for i, (string, df, cf) in terms], density=True)
plt.title('Document Frequency')
plt.show()

In [ ]:
plt.hist([cf for i, (string, df, cf) in terms], density=True)
plt.title('Collection Frequency')
plt.show()

### Feature Selection

In [ ]:
for i, x in enumerate(dataset.x):
    if len(x.shape) != 2:
        print(i, dataset.inds[i], x.shape)

In [ ]:
dictionary = max_seg_chi2(terms, dataset, size=150)

In [ ]:
clf = Classifier(dictionary)

In [ ]:
ma_train_f1, mi_train_f1, ma_valid_f1, mi_valid_f1 = cross_validation(clf, dataset, seed=1126)

In [ ]:
print(f'macro f1 (train): {ma_train_f1}')
print(f'micro f1 (train): {mi_train_f1}')
print(f'macro f1 (valid): {ma_valid_f1}')
print(f'micro f1 (valid): {mi_valid_f1}')

### Confusion Matrix

In [ ]:
def confusion_matrix(clf, dataset, fold=10, permutation=True, seed=None):
  if seed:
    np.random.seed(seed)
    random.seed(seed)
  n = len(dataset)
  if permutation:
    permute = np.random.permutation(n)
  else:
    permute = np.arange(n)
  width = int(np.ceil(n / fold))
  left = 0
  right = width
  mat = np.zeros((4, 4))
  for i in range(fold):
    train_permute = np.concatenate((permute[:left], permute[right:]))
    test_permute = permute[left:right]
#     test_permute = np.concatenate((permute[:left], permute[right:]))
#     train_permute = permute[left:right]
    left += width
    right += width
    dataset_train, dataset_test = train_test_split(dataset, train_permute, test_permute)
    clf.train(dataset_train)
    pred = clf.predict(dataset_test)
#     print(min(dataset_test.y), max(dataset_test.y))
    for i in range(len(dataset_test)):
        mat[pred[i,1]-2, int(dataset_test.y[i]-2)] += 1
  mat /= fold
  print(mat)
  return mat

In [ ]:
mat = confusion_matrix(clf, dataset)

- safe < euclid < keter
- thaumiel
- neutralized

In [ ]:
import seaborn as sns
print(fact2[1])
sns.heatmap(mat, annot=True,  linewidths=.5)
plt.show()

In [ ]:
print('Precision')
print(fact2[1])
print(np.diag(mat) / np.sum(mat, axis=1))

In [ ]:
print('Recall')
print(fact2[1])
print(np.diag(mat) / np.sum(mat, axis=0))

In [ ]:
print('F1')
print(fact2[1])
p = np.diag(mat) / np.sum(mat, axis=1)
r = np.diag(mat) / np.sum(mat, axis=0)
f = 2 * p * r / (p + r)
print(f)

In [ ]:
fig, ax = plt.subplots()

x = np.arange(len(fact2[1]))

width = 0.2
rects1 = ax.bar(x - width, p, width, label='Precision')
rects2 = ax.bar(x, r, width, label='Recall')
rects1 = ax.bar(x + width, f, width, label='F1')

ax.set_ylabel('Scores')
ax.set_title('Scores by class')
ax.set_xticks(x)
ax.set_xticklabels(fact2[1])
ax.legend()

plt.show()

In [ ]:
sum(df.object_class == 'thaumiel')

### Check the selected terms

In [ ]:
np.array(terms, dtype=object)[np.array(dictionary)-1]